In [15]:
import math
import numpy as np

import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, iplot

In [16]:
def get_fractured(source, min_points=1, max_points=4, min_radius=3, max_radius=6, sphere_chance=0.75):
    fractured = source.copy()
    points = np.random.randint(min_points, max_points+1)
    idxs = np.argwhere(fractured == 1)
    centers = idxs[np.random.choice(len(idxs), points, False)]
    
    for x,y,z in centers:
        r = np.random.randint(min_radius, max_radius+1)
        xmin, xmax = max(0, x-r), x+r
        ymin, ymax = max(0, y-r), y+r
        zmin, zmax = max(0, z-r), z+r
        sphere = np.ones_like(fractured)
        sphere[xmin:xmax, ymin:ymax, zmin:zmax] = 0
        # sphere or cube
        if np.random.rand() < sphere_chance:
            idxs = np.argwhere(sphere == 0)
            idxs_remove = np.sqrt((idxs[:,0] - x)**2 + (idxs[:,1] - y)**2 + (idxs[:,2] - z)**2)
            idxs_remove = idxs[idxs_remove > r]
            sphere[idxs_remove[:,0], idxs_remove[:,1], idxs_remove[:,2]] = 1
            
        fractured *= sphere
    
    return fractured

In [17]:
def volume_to_point_cloud(vol):
    """ vol is occupancy grid (value = 0 or 1) of size vsize*vsize*vsize
        return Nx3 numpy array.
    """
    vsize = vol.shape[0]
    assert(vol.shape[1] == vsize and vol.shape[1] == vsize)
    points = []
    for a in range(vsize):
        for b in range(vsize):
            for c in range(vsize):
                if vol[a,b,c] == 1:
                    points.append(np.array([a,b,c]))
    if len(points) == 0:
        return np.zeros((0,3))
    points = np.vstack(points)
    
    return points

In [18]:
def plot_vol(vol, s=10, c=(105,127,155), show_grid=False):
    if vol.dtype != np.bool:
        vol = vol > 0

    pc = volume_to_point_cloud(vol)
    plot3d(pc, s, c, show_grid)

In [6]:
data = np.load('./modelnet10.npy').item()['test']['data']

ValueError: can only convert an array of size 1 to a Python scalar

In [7]:
data = np.load('./modelnet10.npy')

In [9]:
data.item()

ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
data = np.load('./custom_arq_dataset.npy')

In [5]:
pip install binvox

  Created wheel for binvox: filename=binvox-0.1.5-py3-none-any.whl size=5174 sha256=157c640d70fe12f25c64a2e79bdbd163c3dcc4fb0cdfabe02cecfb085eb0ccb8
  Stored in directory: c:\users\brebe\appdata\local\pip\cache\wheels\90\60\92\4a1371a804a6a54bf33cd2be960651cf09a001b1e7a209077c
Successfully built binvox
Note: you may need to restart the kernel to use updated packages.


In [27]:
import os
from collections import Counter
import random

In [40]:
path = './ModelNet/ModelNet10'
labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
files = [os.path.join(path,l,'train',ll) for l in labels
         for ll in os.listdir(os.path.join(path, l, 'train'))
         if ll[-4:] == '.off']

print(labels)
print(Counter([f.split('./ModelNet/ModelNet10')[1].split('\\')[1] for f in files]))

['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
Counter({'chair': 889, 'sofa': 680, 'bed': 515, 'monitor': 465, 'table': 392, 'toilet': 344, 'desk': 200, 'dresser': 200, 'night_stand': 200, 'bathtub': 106})



Plotting: ./ModelNet/ModelNet10\sofa\train\sofa_0177.off


AttributeError: 'NoneType' object has no attribute 'dtype'

In [29]:
import zipfile as zf

In [31]:
files = zf.ZipFile("./ModelNet/ModelNet40.zip", 'r')
files.extractall('./ModelNet/')
files.close()

In [39]:
files = zf.ZipFile("./ModelNet/ModelNet10.zip", 'r')
files.extractall('./ModelNet/')
files.close()

In [44]:
from time import time

In [46]:
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

def multithreading(func, args, workers):
    with ThreadPoolExecutor(max_workers=workers) as executor:
        res = executor.map(func, args)
    return list(res)

get_label = lambda x: x.split('ModelNet')[1][3:].split('/')[0]

def get_voxels(files, voxsize):
    data = np.ndarray((0, *[voxsize]*3), dtype=np.bool)
    labels = []
    errors = []
    
    for i, file in enumerate(files):
        res = voxels_from_file(file, voxsize)
        if res[0] == 1:
            labels.append(get_label(file))
            data = np.vstack([data, res[1].reshape((1, *res[1].shape))])
        else:
            errors.append(file)

    return labels, data, errors

get_voxels_parallel = lambda x: get_voxels(*x)

def convert_all(path, voxsize):
    out_file = os.path.join(path, 'voxels.npy')
    labels = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    train_files = [os.path.join(path,l,'train',ll) for l in labels
                   for ll in os.listdir(os.path.join(path, l, 'train'))
                   if ll[-4:] == '.off']
    test_files = [os.path.join(path,l,'test',ll) for l in labels
                   for ll in os.listdir(os.path.join(path, l, 'test'))
                   if ll[-4:] == '.off']

    print(f'train: {len(train_files)}, test: {len(test_files)}')
    
    n_cpu = multiprocessing.cpu_count()
    output = {}
    
    for data_files, data_name in zip([train_files, test_files], ['train', 'test']):
        t0 = time()
        print(f'Launching {n_cpu} threads for {data_name} set...', end='')
        thread_size = math.ceil(len(data_files) / n_cpu)
        args = [(data_files[i*thread_size:(i+1)*thread_size], voxsize) for i in range(n_cpu)]
        res = multithreading(get_voxels_parallel, args, n_cpu)
        labels = []
        data = np.ndarray((0, *[voxsize]*3), dtype=np.bool)
        errors = []

        for l, d, e in res:
            labels += l
            data = np.vstack([data, d])
            errors += e
            
        output[data_name] = {'labels': labels, 'data': data, 'errors': errors}
        
        print('(%.2fs)' % (time() - t0))
    
    np.save(out_file, output)    
    print('\nSaved on: %s (%.2fM)' % (out_file, os.path.getsize(out_file) / 1024**2))

In [47]:
convert_all('./ModelNet/ModelNet10/', voxsize=32)

train: 3991, test: 908
Launching 16 threads for train set...

C:\Users\brebe\AppData\Local\Temp\ipykernel_31536\4063895973.py:12: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\brebe\AppData\Local\Temp\ipykernel_31536\4063895973.py:50: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



(43.73s)
Launching 16 threads for test set...(11.67s)

Saved on: ./ModelNet/ModelNet10/voxels.npy (0.27M)


In [50]:
data = np.load('./ModelNet/ModelNet10/voxels.npy').item()['test']['data']

ValueError: Object arrays cannot be loaded when allow_pickle=False